In [57]:
import pandas as pd
from cleantext import clean
from sklearn.feature_extraction.text import TfidfVectorizer

In [59]:

xls = pd.ExcelFile("C:\\Users\\bfire\OneDrive\\Desktop\\CSUSpring2025\\Data Mining\\Lab2\\10UnionAddresses_TF_IDF_DSA460_Lab2.xlsx")

sheet_names = xls.sheet_names
df = pd.read_excel(xls, sheet_name=sheet_names[0])  # Load first sheet
df.head()


<>:1: SyntaxWarning: invalid escape sequence '\O'
<>:1: SyntaxWarning: invalid escape sequence '\O'
C:\Users\bfire\AppData\Local\Temp\ipykernel_21824\1697439697.py:1: SyntaxWarning: invalid escape sequence '\O'
  xls = pd.ExcelFile("C:\\Users\\bfire\OneDrive\\Desktop\\CSUSpring2025\\Data Mining\\Lab2\\10UnionAddresses_TF_IDF_DSA460_Lab2.xlsx")


,DSA460/CIS492/CIS593 Data Mining,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,George Washington,October-25-1791,https://www.infoplease.com/homework-help/us-d...,C:Usersdizad01DownloadsGeorge-Washington-Octo...,Fellow-Citizens of the Senate and House of Re...
1,George Washington,November-6-1792,https://www.infoplease.com/homework-help/us-d...,C:Usersdizad01DownloadsGeorge-Washington-Nove...,Fellow-Citizens of the Senate and House of Re...
2,George Washington,December-8-1795,https://www.infoplease.com/homework-help/us-d...,C:Usersdizad01DownloadsGeorge-Washington-Dece...,Fellow-Citizens of the Senate and House of Re...
3,John Adams,November-22-1797,https://www.infoplease.com/homework-help/us-d...,C:Usersdizad01DownloadsJohn-Adams-November-22...,Gentlemen of the Senate and Gentlemen of the ...
4,Franklin Roosevelt,January-3-1936,https://www.infoplease.com/homework-help/us-d...,C:Usersdizad01DownloadsFranklin-Roosevelt-Jan...,Mr. President; Mr. Speaker; Members of the Se...


In [61]:
# Rename relevant columns for clarity
df_clean = df.iloc[:, [0, 1, 4]].copy()
df_clean.columns = ["President", "Date", "Speech_Text"]

# Display the first few rows
df_clean.head()


,President,Date,Speech_Text
0,George Washington,October-25-1791,Fellow-Citizens of the Senate and House of Re...
1,George Washington,November-6-1792,Fellow-Citizens of the Senate and House of Re...
2,George Washington,December-8-1795,Fellow-Citizens of the Senate and House of Re...
3,John Adams,November-22-1797,Gentlemen of the Senate and Gentlemen of the ...
4,Franklin Roosevelt,January-3-1936,Mr. President; Mr. Speaker; Members of the Se...


In [63]:
def preprocess_text_cleantext(text):
    return clean(text,
                 lower=True,                # Convert to lowercase
                 no_punct=True,             # Remove punctuation
                 no_urls=True,              # Remove URLs
                 no_emails=True,            # Remove emails
                 no_numbers=True,           # Remove numbers
                 no_currency_symbols=True,  # Remove currency symbols
                 replace_with_punct="",     # Ensure punctuation is removed
                 replace_with_url="",       # Ensure URLs are removed
                 replace_with_number="",    # Ensure numbers are removed
                 replace_with_currency_symbol="")  # Ensure currency symbols are removed

In [65]:
df_clean["Processed_Speech"] = df_clean["Speech_Text"].apply(preprocess_text_cleantext)

# Display processed text sample
df_clean.head()

,President,Date,Speech_Text,Processed_Speech
0,George Washington,October-25-1791,Fellow-Citizens of the Senate and House of Re...,fellowcitizens of the senate and house of repr...
1,George Washington,November-6-1792,Fellow-Citizens of the Senate and House of Re...,fellowcitizens of the senate and house of repr...
2,George Washington,December-8-1795,Fellow-Citizens of the Senate and House of Re...,fellowcitizens of the senate and house of repr...
3,John Adams,November-22-1797,Gentlemen of the Senate and Gentlemen of the ...,gentlemen of the senate and gentlemen of the h...
4,Franklin Roosevelt,January-3-1936,Mr. President; Mr. Speaker; Members of the Se...,mr president mr speaker members of the senate ...


In [67]:
# Define a basic set of stop words manually
custom_stop_words = set([
    "the", "and", "is", "in", "to", "of", "it", "for", "on", "with", "as", "this",
    "that", "at", "by", "an", "be", "from", "or", "which", "are", "was", "but", "not",
    "can", "if", "we", "you", "your", "our", "has", "have", "will", "would", "should"
])
customTopics = ["freedom", "freedom of speech", "freedom of press",
          "security", "peace", "reestablishment of peace", "preservation of peace"]
vectorizer = TfidfVectorizer(vocabulary=customTopics, ngram_range=(1,3))

# Compute TF-IDF matrix (assuming we have a DataFrame with multiple speeches)
tfidf_matrix = vectorizer.fit_transform([processed_speech])

# Convert to a DataFrame for better readability
import pandas as pd
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Display the scores
print(tfidf_df)

NameError: name 'processed_speech' is not defined

In [51]:

def custom_cosine_similarity(vec1, vec2):
    """Compute cosine similarity manually between two vectors."""
    dot_product = np.dot(vec1, vec2)  # Compute the dot product
    norm_vec1 = np.linalg.norm(vec1)  # Compute magnitude of vec1
    norm_vec2 = np.linalg.norm(vec2)  # Compute magnitude of vec2
    
    # Prevent division by zero
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0
    
    return dot_product / (norm_vec1 * norm_vec2)


In [53]:
num_speeches = tfidf_matrix.shape[0]
topic_vector = np.ones(tfidf_matrix.shape[1])  # Example topic vector (can be refined)

# Compute similarity between each speech and the topic
similarity_scores = [custom_cosine_similarity(tfidf_matrix[i, :], topic_vector) for i in range(num_speeches)]

# Rank speeches by similarity
ranked_speeches = sorted(zip(similarity_scores, df_clean["President"]), reverse=True)
print("Top speeches discussing the topic:", ranked_speeches[:5])

NameError: name 'tfidf_matrix' is not defined